In [1]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier

import os
import re
import gc
import warnings
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
sectors = ['finance', 'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy', 'electrical networks', 'telecom', 'it', 'real_estate']
companies = ['ALRS', 'AFLT', 'VTBR', 'GAZP', 'GMKN', 'LSRG', 'DSKY', 'IRAO', 'LKOH', 'MAGN', 'MOEX', 'CBOM', 'MTSS', 'ROSN', 'NLMK', 'NVTK', 'PIKK', 'PLZL',
              'RTKM', 'RUAL', 'HYDR', 'SBER', 'CHMF', 'AFKS', 'SNGS', 'TATN', 'TCSG', 'TRNFP', 'POGR', 'PHOR', 'FEES', 'HHRU', 'YNDX']

## Телеграмм

In [ ]:
tg = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg.parquet')
# tg_raw = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg_raw.parquet')
# tg.drop(columns=[x for x in tg.columns if bool(re.search('_close', x))], inplace=True)
# tg = tg.iloc[:, :2].copy()
# tg = tg.merge(tg_raw, how='left', on='date')
# del tg_raw
tg.dropna(subset=['message', 'date'], inplace=True)

In [ ]:
tg = tg.drop(columns=['views', 'forwards', 'fwd_from'])
tg = tg[tg.date < '2022-01-01'].copy()
# tg = tg.drop(columns=sectors)
# tg.rename(columns={'message_x': 'message_prep', 'message_y': 'message_raw'}, inplace=True)

In [ ]:
# Случайный лес
targets = tg.iloc[:,3:-1].columns
for company in companies:
  print(company)
  os.makedirs(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}', exist_ok=True)
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv', nrows=3).date[0]
  comp = tg[tg[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
  tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2))
  features_train = tfidf.fit_transform(train['message'])
  features_test = tfidf.transform(test['message'])
  X_train = features_train.todense()
  X_test = features_test.todense()
  tr = [col for col in targets if company in col]
  train_logg = []
  test_logg = []
  base_acc = []
  for t in tr:
    y_train = train[t].values.astype(int)
    y_test = test[t].values.astype(int)
    base_acc.append(np.mean(y_train))
    for depth in range(3, 13):
      clf = RandomForestClassifier(random_state=123, criterion='entropy', n_jobs=-1, max_depth=depth)
      y_pred = clf.fit(np.asarray(X_train), y_train).predict(np.asarray(X_test))
      test_logg.append(np.mean(y_test==y_pred))
      train_pred = clf.predict(np.asarray(X_train))
      train_logg.append(np.mean(y_train==train_pred))

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_train_logg.txt', 'w') as fp:
    for item in train_logg:
        fp.write("%s\n" % item)
  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_test_logg.txt', 'w') as fp:
    for item in test_logg:
        fp.write("%s\n" % item)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_base_acc_logg.txt', 'w') as fp:
    for item in base_acc:
        fp.write("%s\n" % item)

  
  print(f'{company} done')

In [ ]:
#Градиентный бустинг
targets = tg.iloc[:,3:-1].columns
for company in companies:
  print(company)
  # os.mkdir(f'/content/drive/MyDrive/Диссертация/{company}')
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv', nrows=3).date[0]
  comp = tg[tg[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
  tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2))
  features_train = tfidf.fit_transform(train['message'])
  features_test = tfidf.transform(test['message'])
  X_train = features_train.todense()
  X_test = features_test.todense()
  tr = [col for col in targets if company in col]
  train_logg = []
  test_logg = []
  base_acc = []
  for t in tr:
    y_train = train[t].values.astype(int)
    y_test = test[t].values.astype(int)
    base_acc.append(np.mean(y_train))
    for depth in range(3, 13):
      clf = LGBMClassifier(random_state=123, n_jobs=-1, max_depth=depth)
      y_pred = clf.fit(np.asarray(X_train), y_train).predict(np.asarray(X_test))
      test_logg.append(np.mean(y_test==y_pred))
      train_pred = clf.predict(np.asarray(X_train))
      train_logg.append(np.mean(y_train==train_pred))

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_train_logg.txt', 'w') as fp:
    for item in train_logg:
        fp.write("%s\n" % item)
  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_test_logg.txt', 'w') as fp:
    for item in test_logg:
        fp.write("%s\n" % item)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_base_acc_logg.txt', 'w') as fp:
    for item in base_acc:
        fp.write("%s\n" % item)

  
  print(f'{company} done')

### Сведение результатов в таблицу

In [ ]:
import pandas as pd
import numpy as np

In [15]:
results_rf_tg = pd.DataFrame(columns=['1 min test', '1 min train', 'base 1 min', '5 min test', '5 min train', 'base 5 min', '10 min test', '10 min train', 'base 10 min', '15 min test', '15 min train', 'base 15 min',
                                   '30 min test', '30 min train', 'base 30 min', '1 hour test', '1 hour train', 'base 1 hour', '1 day test', '1 day train', 'base 1 day'], index=companies)

In [16]:
for comp in companies:
  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_rf_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 10)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_rf_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 10)
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_rf_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]

  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba in zip(lines_train, lines_test, base_train):
    res.append(te)
    res.append(tr)
    res.append(ba)
  
  if len(res) == 24:
    res = res[3:]

  results_rf_tg.loc[comp,:] = res

In [17]:
pd.set_option('display.max_columns', 21)
pd.set_option('display.max_rows', 33)

In [18]:
results_rf_tg

,1 min test,1 min train,base 1 min,5 min test,5 min train,base 5 min,10 min test,10 min train,base 10 min,15 min test,15 min train,base 15 min,30 min test,30 min train,base 30 min,1 hour test,1 hour train,base 1 hour,1 day test,1 day train,base 1 day
ALRS,0.542493,0.630982,0.509748,0.545326,0.702588,0.506558,0.582153,0.676001,0.517901,0.484419,0.772776,0.49805,0.46034,0.79369,0.490961,0.597734,0.638072,0.510457,0.563739,0.577455,0.515066
AFLT,0.507675,0.739857,0.511239,0.512061,0.765077,0.503564,0.517544,0.708882,0.495614,0.509868,0.725877,0.501096,0.539474,0.676809,0.501371,0.527412,0.706414,0.49205,0.529605,0.578125,0.448191
VTBR,0.57905,0.534521,0.534521,0.57243,0.533937,0.53384,0.567368,0.547278,0.528971,0.577492,0.539975,0.532184,0.541667,0.549031,0.53569,0.523754,0.535593,0.535593,0.509346,0.68809,0.495277
GAZP,0.557003,0.542331,0.517131,0.550963,0.531524,0.528976,0.537939,0.549976,0.51798,0.544356,0.517603,0.513119,0.555304,0.520434,0.515007,0.56191,0.524021,0.519066,0.573424,0.528032,0.528032
GMKN,0.47541,0.573955,0.488336,0.493852,0.791079,0.503461,0.509221,0.721097,0.49859,0.5,0.788772,0.500641,0.495902,0.634196,0.4909,0.516393,0.826455,0.505255,0.570697,0.687003,0.493207
LSRG,0.55,0.868715,0.511173,0.6,0.723464,0.530726,0.561111,0.776536,0.546089,0.583333,0.833799,0.52095,0.555556,0.824022,0.536313,0.588889,0.877095,0.527933,0.55,0.833799,0.469274
DSKY,0.637333,0.604947,0.570856,0.634667,0.590909,0.583556,0.656,0.695187,0.575535,0.666667,0.620989,0.56484,0.645333,0.671123,0.575535,0.629333,0.723262,0.555481,0.512,0.68115,0.538102
IRAO,0.590799,0.657177,0.508783,0.542373,0.855845,0.509388,0.506053,0.705633,0.517262,0.520581,0.61599,0.522108,0.544794,0.823743,0.514839,0.523002,0.708661,0.505754,0.501211,0.579043,0.460933
LKOH,0.553138,0.664295,0.510674,0.524686,0.810381,0.500419,0.525523,0.651946,0.488489,0.524686,0.735245,0.490163,0.514644,0.790917,0.500628,0.516318,0.668271,0.502093,0.56569,0.595228,0.510255
MAGN,0.516411,0.690241,0.552632,0.492341,0.729715,0.526864,0.507659,0.662829,0.528509,0.50547,0.708882,0.552632,0.492341,0.744518,0.544956,0.49453,0.729715,0.531798,0.579869,0.699561,0.482456


In [19]:
results_boosting_tg = pd.DataFrame(columns=['1 min test', '1 min train', 'base 1 min', '5 min test', '5 min train', 'base 5 min', '10 min test', '10 min train', 'base 10 min', '15 min test', '15 min train', 'base 15 min',
                                         '30 min test', '30 min train', 'base 30 min', '1 hour test', '1 hour train', 'base 1 hour', '1 day test', '1 day train', 'base 1 day'], index=companies)

In [ ]:
for comp in companies:
  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_boosting_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 10)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_boosting_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 10)
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{comp}/{comp}_boosting_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]

  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba in zip(lines_train, lines_test, base_train):
    res.append(te)
    res.append(tr)
    res.append(ba)

  if len(res) == 24:
    res = res[3:]

  results_boosting_tg.loc[comp,:] = res

In [ ]:
results_boosting_tg

,1 min test,1 min train,base 1 min,5 min test,5 min train,base 5 min,10 min test,10 min train,base 10 min,15 min test,15 min train,base 15 min,30 min test,30 min train,base 30 min,1 hour test,1 hour train,base 1 hour,1 day test,1 day train,base 1 day
ALRS,0.531161,0.832329,0.509748,0.522663,0.766395,0.506558,0.549575,0.74229,0.517901,0.505666,0.894718,0.49805,0.526912,0.86955,0.490961,0.549575,0.748671,0.510457,0.521246,0.756115,0.515066
AFLT,0.528509,0.864857,0.511239,0.525219,0.845121,0.503564,0.506579,0.866502,0.495614,0.501096,0.780702,0.501096,0.51864,0.840461,0.501371,0.512061,0.792215,0.49205,0.549342,0.723958,0.448191
VTBR,0.561526,0.613789,0.534521,0.544393,0.617587,0.53384,0.541667,0.632194,0.528971,0.556854,0.631512,0.532184,0.537773,0.631415,0.53569,0.530763,0.65761,0.535593,0.507399,0.760249,0.495277
GAZP,0.530011,0.661916,0.517131,0.544356,0.600472,0.528976,0.539638,0.652336,0.51798,0.535674,0.599009,0.513119,0.540204,0.595092,0.515007,0.548509,0.596225,0.519066,0.553039,0.605757,0.528032
GMKN,0.5,0.862856,0.488336,0.506148,0.864394,0.503461,0.514344,0.747757,0.49859,0.519467,0.879518,0.500641,0.532787,0.831582,0.4909,0.529713,0.83594,0.505255,0.548156,0.738529,0.493207
LSRG,0.511111,0.797486,0.511173,0.561111,0.861732,0.530726,0.522222,0.905028,0.546089,0.538889,0.856145,0.52095,0.594444,0.912011,0.536313,0.5,0.863128,0.527933,0.522222,0.850559,0.469274
DSKY,0.64,0.809492,0.570856,0.621333,0.817513,0.583556,0.632,0.899733,0.575535,0.632,0.777406,0.56484,0.616,0.782754,0.575535,0.624,0.910428,0.555481,0.533333,0.802139,0.538102
IRAO,0.569007,0.821926,0.508783,0.535109,0.897638,0.509388,0.501211,0.82132,0.517262,0.508475,0.904906,0.522108,0.547215,0.783162,0.514839,0.469734,0.849182,0.505754,0.493947,0.903695,0.460933
LKOH,0.519665,0.787149,0.510674,0.515481,0.802428,0.500419,0.50795,0.719757,0.488489,0.506276,0.8018,0.490163,0.517155,0.757639,0.500628,0.533891,0.795312,0.502093,0.541423,0.825659,0.510255
MAGN,0.531729,0.872259,0.552632,0.557987,0.868421,0.526864,0.525164,0.871162,0.528509,0.507659,0.863487,0.552632,0.525164,0.76864,0.544956,0.50547,0.893092,0.531798,0.533917,0.779605,0.482456


## Нейросетевой подход Телеграмм

#### Дообучаем bert

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.6 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.optim import AdamW
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from transformers import get_linear_schedule_with_warmup

tokenizer = AutoTokenizer.from_pretrained("RussianNLP/ruRoBERTa-large-rucola")

In [ ]:
class TgDataset(Dataset):
    def __init__(self, texts, targets):
    
      self.labels = targets.astype(int)
      self.texts = [tokenizer(text, 
                          padding='max_length', max_length = 512, truncation=True,
                          return_tensors="pt") for text in texts]

    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):

        batch_texts = self.texts[idx]
        batch_y = self.labels[idx]

        return batch_texts, batch_y

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, n_classes=7):

        super(BertClassifier, self).__init__()

        self.bert = AutoModelForSequenceClassification.from_pretrained("RussianNLP/ruRoBERTa-large-rucola",
                                                                       num_labels=n_classes,
                                                                       ignore_mismatched_sizes=True)
        # self.classifier = nn.Linear(1024, n_classes)
        # self.act = nn.Softmax(dim=1)

    def forward(self, input_id, mask):

        output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        # output = self.act(output[0])
        return output[0]

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, device, epochs, scheduler=None):

    mean_1_min, mean_5_min, mean_10_min, mean_15_min, mean_30_min, mean_1_hour, mean_1_day = train_target.mean(axis=0)

    model = model.to(device)

    for epoch_num in range(epochs):
        model.train()

        acc_1_min_train = 0
        acc_5_min_train = 0
        acc_10_min_train = 0
        acc_15_min_train = 0
        acc_30_min_train = 0
        acc_1_hour_train = 0
        acc_1_day_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output, train_label.float())
            total_loss_train += batch_loss.item()
            out = torch.sigmoid(output)

            acc_1_min_train += ((out[:,0] > mean_1_min).int() == train_label[:,0]).sum().item()
            acc_5_min_train += ((out[:,1] > mean_5_min).int() == train_label[:,1]).sum().item()
            acc_10_min_train += ((out[:,2] > mean_10_min).int() == train_label[:,2]).sum().item()
            acc_15_min_train += ((out[:,3] > mean_15_min).int() == train_label[:,3]).sum().item()
            acc_30_min_train += ((out[:,4] > mean_30_min).int() == train_label[:,4]).sum().item()
            acc_1_hour_train += ((out[:,5] > mean_1_hour).int() == train_label[:,5]).sum().item()
            acc_1_day_train += ((out[:,6] > mean_1_day).int() == train_label[:,6]).sum().item()

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()
      
        acc_1_min_val = 0
        acc_5_min_val = 0
        acc_10_min_val = 0
        acc_15_min_val = 0
        acc_30_min_val = 0
        acc_1_hour_val = 0
        acc_1_day_val = 0
        total_loss_val = 0

        with torch.no_grad():
            model.eval()
            for val_input, val_label in tqdm(val_dataloader):

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.float())
                total_loss_val += batch_loss.item()
                out = F.softmax(output, dim=-1)

                acc_1_min_val += ((out[:,0] > mean_1_min).int() == val_label[:,0]).sum().item()
                acc_5_min_val += ((out[:,1] > mean_5_min).int() == val_label[:,1]).sum().item()
                acc_10_min_val += ((out[:,2] > mean_10_min).int() == val_label[:,2]).sum().item()
                acc_15_min_val += ((out[:,3] > mean_15_min).int() == val_label[:,3]).sum().item()
                acc_30_min_val += ((out[:,4] > mean_30_min).int() == val_label[:,4]).sum().item()
                acc_1_hour_val += ((out[:,5] > mean_1_hour).int() == val_label[:,5]).sum().item()
                acc_1_day_val += ((out[:,6] > mean_1_day).int() == val_label[:,6]).sum().item()
      
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .5f} \
                | Train Acc 1 min: {acc_1_min_train / len(train_dataset): .5f} \
                | Train Acc 5 min: {acc_5_min_train / len(train_dataset): .5f} \
                | Train Acc 10 min: {acc_10_min_train / len(train_dataset): .5f} \
                | Train Acc 15 min: {acc_15_min_train / len(train_dataset): .5f} \
                | Train Acc 30 min: {acc_30_min_train / len(train_dataset): .5f} \
                | Train Acc 1 hour: {acc_1_hour_train / len(train_dataset): .5f} \
                | Train Acc 1 day: {acc_1_day_train / len(train_dataset): .5f} \
                | Val Loss: {total_loss_val / len(val_dataset): .5f} \
                | Val Acc 1 min: {acc_1_min_val / len(val_dataset): .5f} \
                | Val Acc 5 min: {acc_5_min_val / len(val_dataset): .5f} \
                | Val Acc 10 min: {acc_10_min_val / len(val_dataset): .5f} \
                | Val Acc 15 min: {acc_15_min_val / len(val_dataset): .5f} \
                | Val Acc 30 min: {acc_30_min_val / len(val_dataset): .5f} \
                | Val Acc 1 hour: {acc_1_hour_val / len(val_dataset): .5f} \
                | Val Acc 1 day: {acc_1_day_val / len(val_dataset): .5f}')

In [ ]:
tg = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg_raw.parquet')

In [ ]:
comp = tg[tg['GAZP'] == True].copy()
comp.drop(columns=['views', 'forwards', 'fwd_from', 'source'], inplace=True)
comp.dropna(inplace=True)
comp = comp[comp['date'] < '2022-01-01'].copy()
cols = ['date', 'message'] + [col for col in comp.columns if 'GAZP_' in col]
comp = comp[cols].copy()
train_data, test_data = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
cols = [col for col in comp.columns if 'GAZP_' in col]

In [ ]:
train_texts, val_texts = train_data['message'].values, test_data['message'].values
train_target, val_target = train_data[cols].values, test_data[cols].values

In [ ]:
batch_size = 4
epochs = 3
train_dataset, val_dataset = TgDataset(train_texts, train_target), TgDataset(val_texts, val_target)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
total_steps = len(train_dataloader) * epochs

In [ ]:
n_classes = 7
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BertClassifier(n_classes)
LR = 2e-5
optimizer = AdamW(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(0.1*total_steps),
                                            num_training_steps = total_steps)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at RussianNLP/ruRoBERTa-large-rucola and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([7, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.bert.roberta.requires_grad = False

In [ ]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, device, 1)

100%|██████████| 1281/1281 [09:03<00:00,  2.36it/s]

Epochs: 1 | Train Loss:  0.17584                 | Train Acc 1 min:  0.49717                 | Train Acc 5 min:  0.47540                 | Train Acc 10 min:  0.49000                 | Train Acc 15 min:  0.50347                 | Train Acc 30 min:  0.50107                 | Train Acc 1 hour:  0.51040                 | Train Acc 1 day:  0.48951                 | Val Loss:  0.17463                 | Val Acc 1 min:  0.44388                 | Val Acc 5 min:  0.45071                 | Val Acc 10 min:  0.46438                 | Val Acc 15 min:  0.45813                 | Val Acc 30 min:  0.44603                 | Val Acc 1 hour:  0.43978                 | Val Acc 1 day:  0.42495


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Диссертация/7_cls.pth')

In [ ]:
model.bert.roberta.requires_grad = True

In [ ]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, device, epochs, scheduler)

 89%|████████▊ | 4540/5123 [1:34:33<12:08,  1.25s/it]

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Диссертация/Tg_raw_RuRoberta_gazp.pth')

## Традиционные новости

In [ ]:
news = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/news.parquet')

In [ ]:
news = news[news.date < '2022-01-01'].copy()
news.drop(columns=sectors, inplace=True)

In [ ]:
news['message'] = news['title'] + ' ' + news['announce']
news.drop(columns=['link', 'title', 'announce', 'text'], inplace=True)

In [ ]:
# Случайный лес
targets = news.iloc[:, 2:].columns
for company in companies:
    print(company)
    first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv', nrows=3).date[0]
    print(first_day)
    os.makedirs(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}', exist_ok=True)
    comp = news[news[company] == True].copy()
    comp = comp[comp.date >= first_day].copy()
    comp.dropna(inplace=True)
    train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2))
    features_train = tfidf.fit_transform(train['message'])
    features_test = tfidf.transform(test['message'])
    X_train = features_train.todense()
    X_test = features_test.todense()
    tr = [col for col in targets if company in col]
    if company in tr:
      tr.remove(company)
    train_logg = []
    test_logg = []
    base_acc = []
    for t in tr:
        y_train = train[t].values.astype(int)
        y_test = test[t].values.astype(int)
        base_acc.append(np.mean(y_train))
        for depth in range(3, 16):
            clf = RandomForestClassifier(random_state=123, criterion='entropy', n_jobs=-1, max_depth=depth)
            y_pred = clf.fit(np.asarray(X_train), y_train).predict(np.asarray(X_test))
            test_logg.append(np.mean(y_test==y_pred))
            train_pred = clf.predict(np.asarray(X_train))
            train_logg.append(np.mean(y_train==train_pred))

        with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_train_logg.txt', 'w') as fp:
          for item in train_logg:
            fp.write("%s\n" % item)
        with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_test_logg.txt', 'w') as fp:
          for item in test_logg:
            fp.write("%s\n" % item)

        with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_base_acc_logg.txt', 'w') as fp:
          for item in base_acc:
            fp.write("%s\n" % item)


        print(f'{company} done')

In [ ]:
#Градиентный бустинг
targets = news.iloc[:, 2:].columns
for company in companies:
  print(company)
  # os.mkdir(f'/content/drive/MyDrive/Диссертация/{company}')
  comp = news[news[company] == True].copy()
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv', nrows=3).date[0]
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
  tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2))
  features_train = tfidf.fit_transform(train['message'])
  features_test = tfidf.transform(test['message'])
  X_train = features_train.todense()
  X_test = features_test.todense()
  tr = [col for col in targets if company in col]
  if company in tr:
    tr.remove(company)
  train_logg = []
  test_logg = []
  base_acc = []
  for t in tr:
    y_train = train[t].values.astype(int)
    y_test = test[t].values.astype(int)
    base_acc.append(np.mean(y_train))
    for depth in range(3, 16):
      clf = LGBMClassifier(random_state=123, n_jobs=-1, max_depth=depth)
      y_pred = clf.fit(np.asarray(X_train), y_train).predict(np.asarray(X_test))
      test_logg.append(np.mean(y_test==y_pred))
      train_pred = clf.predict(np.asarray(X_train))
      train_logg.append(np.mean(y_train==train_pred))

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_train_logg.txt', 'w') as fp:
    for item in train_logg:
        fp.write("%s\n" % item)
  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_test_logg.txt', 'w') as fp:
    for item in test_logg:
        fp.write("%s\n" % item)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_base_acc_logg.txt', 'w') as fp:
    for item in base_acc:
        fp.write("%s\n" % item)

  
  print(f'{company} done')

In [ ]:
results_rf_news = pd.DataFrame(columns=['1 min test', '1 min train', 'base 1 min', '5 min test', '5 min train', 'base 5 min', '10 min test', '10 min train', 'base 10 min', '15 min test', '15 min train', 'base 15 min',
                                   '30 min test', '30 min train', 'base 30 min', '1 hour test', '1 hour train', 'base 1 hour', '1 day test', '1 day train', 'base 1 day'], index=companies)

In [ ]:
for comp in companies:
  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_rf_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 13)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_rf_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 13)
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_rf_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]

  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba in zip(lines_train, lines_test, base_train):
    res.append(te)
    res.append(tr)
    res.append(ba)
  
  if len(res) == 24:
    res = res[3:]

  results_rf_news.loc[comp,:] = res

In [ ]:
pd.set_option('display.max_columns', 21)
pd.set_option('display.max_rows', 33)

In [ ]:
results_rf_news

,1 min test,1 min train,base 1 min,5 min test,5 min train,base 5 min,10 min test,10 min train,base 10 min,15 min test,15 min train,base 15 min,30 min test,30 min train,base 30 min,1 hour test,1 hour train,base 1 hour,1 day test,1 day train,base 1 day
ALRS,0.517316,0.865041,0.504065,0.497835,0.834688,0.515989,0.480519,0.78374,0.524661,0.502165,0.747967,0.523577,0.497835,0.678591,0.530081,0.504329,0.704607,0.538211,0.508658,0.723035,0.524661
AFLT,0.533918,0.736981,0.505851,0.519298,0.572557,0.535693,0.523977,0.540521,0.52677,0.527485,0.544617,0.520187,0.506433,0.527355,0.523844,0.503509,0.560854,0.532621,0.492398,0.740638,0.511703
VTBR,0.536398,0.525844,0.521998,0.544841,0.542542,0.539493,0.546904,0.540009,0.531379,0.550657,0.534662,0.529362,0.554034,0.540478,0.524343,0.545403,0.523874,0.518621,0.508255,0.617871,0.500797
GAZP,0.52203,0.528197,0.525803,0.533963,0.53177,0.531443,0.527406,0.535082,0.534066,0.519801,0.529475,0.529475,0.521768,0.537279,0.535574,0.522292,0.536689,0.53423,0.525046,0.609344,0.50118
GMKN,0.49348,0.58143,0.537767,0.509529,0.572898,0.555082,0.509529,0.586951,0.563864,0.512538,0.593225,0.55458,0.484453,0.573149,0.554831,0.48345,0.605772,0.552823,0.519559,0.577666,0.529235
LSRG,0.381818,0.798627,0.475973,0.581818,0.71167,0.519451,0.5,0.80778,0.524027,0.618182,0.679634,0.528604,0.445455,0.720824,0.514874,0.609091,0.620137,0.540046,0.609091,0.590389,0.562929
DSKY,0.616667,0.658333,0.579167,0.65,0.6375,0.6,0.616667,0.916667,0.6125,0.533333,0.695833,0.616667,0.55,0.941667,0.620833,0.533333,0.679167,0.595833,0.583333,0.829167,0.5
IRAO,0.504777,0.7045,0.532457,0.511146,0.632417,0.581043,0.506369,0.590203,0.574273,0.504777,0.590601,0.564715,0.488854,0.606531,0.57507,0.488854,0.633214,0.571087,0.482484,0.733174,0.52728
LKOH,0.505039,0.580031,0.513934,0.524076,0.573729,0.517294,0.506719,0.526537,0.513654,0.515677,0.521916,0.516174,0.513438,0.546282,0.507772,0.511758,0.534799,0.515614,0.493841,0.552164,0.507352
MAGN,0.54,0.727989,0.538296,0.56,0.571224,0.559771,0.528571,0.670007,0.55476,0.54,0.603436,0.551181,0.537143,0.632069,0.559771,0.54,0.554044,0.554044,0.557143,0.710809,0.559771


In [ ]:
results_boosting_news = pd.DataFrame(columns=['1 min test', '1 min train', 'base 1 min', '5 min test', '5 min train', 'base 5 min', '10 min test', '10 min train', 'base 10 min', '15 min test', '15 min train', 'base 15 min',
                                         '30 min test', '30 min train', 'base 30 min', '1 hour test', '1 hour train', 'base 1 hour', '1 day test', '1 day train', 'base 1 day'], index=companies)

In [ ]:
for comp in companies:
  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_boosting_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 13)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_boosting_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 13)
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{comp}/{comp}_boosting_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]

  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba in zip(lines_train, lines_test, base_train):
    res.append(te)
    res.append(tr)
    res.append(ba)

  if len(res) == 24:
    res = res[3:]

  results_boosting_news.loc[comp,:] = res

In [ ]:
results_boosting_news

,1 min test,1 min train,base 1 min,5 min test,5 min train,base 5 min,10 min test,10 min train,base 10 min,15 min test,15 min train,base 15 min,30 min test,30 min train,base 30 min,1 hour test,1 hour train,base 1 hour,1 day test,1 day train,base 1 day
ALRS,0.521645,0.784824,0.504065,0.491342,0.84878,0.515989,0.493506,0.838482,0.524661,0.493506,0.874255,0.523577,0.515152,0.837398,0.530081,0.515152,0.792954,0.538211,0.517316,0.868293,0.524661
AFLT,0.524561,0.70275,0.505851,0.507018,0.625366,0.535693,0.524561,0.743125,0.52677,0.524561,0.76141,0.520187,0.494152,0.668373,0.523844,0.497076,0.62361,0.532621,0.518129,0.63268,0.511703
VTBR,0.529831,0.593293,0.521998,0.534897,0.574437,0.539493,0.540338,0.571764,0.531379,0.543152,0.576407,0.529362,0.547655,0.588931,0.524343,0.529268,0.584193,0.518621,0.509381,0.580675,0.500797
GAZP,0.521374,0.567574,0.525803,0.534618,0.56141,0.531443,0.523735,0.564295,0.534066,0.520063,0.662525,0.529475,0.516916,0.564852,0.535574,0.520456,0.591443,0.53423,0.516654,0.677967,0.50118
GMKN,0.519559,0.81079,0.537767,0.513541,0.697365,0.555082,0.521565,0.764366,0.563864,0.51655,0.728231,0.55458,0.48345,0.806524,0.554831,0.491474,0.75207,0.552823,0.521565,0.658971,0.529235
LSRG,0.454545,0.867277,0.475973,0.445455,0.828375,0.519451,0.518182,0.773455,0.524027,0.490909,0.805492,0.528604,0.436364,0.869565,0.514874,0.454545,0.771167,0.540046,0.563636,0.858124,0.562929
DSKY,0.55,0.783333,0.579167,0.583333,0.770833,0.6,0.566667,0.758333,0.6125,0.566667,0.816667,0.616667,0.533333,0.783333,0.620833,0.433333,0.75,0.595833,0.433333,0.729167,0.5
IRAO,0.515924,0.824771,0.532457,0.528662,0.717244,0.581043,0.533439,0.703305,0.574273,0.512739,0.767822,0.564715,0.522293,0.819594,0.57507,0.549363,0.847471,0.571087,0.55414,0.818001,0.52728
LKOH,0.514558,0.662232,0.513934,0.513998,0.765579,0.517294,0.5028,0.749475,0.513654,0.508959,0.672455,0.516174,0.511198,0.65257,0.507772,0.529115,0.733231,0.515614,0.512318,0.792606,0.507352
MAGN,0.511429,0.70723,0.538296,0.511429,0.69864,0.559771,0.522857,0.85612,0.55476,0.502857,0.811739,0.551181,0.54,0.748031,0.559771,0.531429,0.705798,0.554044,0.54,0.704366,0.559771


## Тесты на значимость полученных результатов

In [22]:
def sample_test(prob1, test_values, n_times=1000):
  accs = []
  for _ in range(n_times):
    rand_test = np.random.choice([0, 1], size=len(test_values), replace=True, p=[1-prob1, prob1])
    acc = (test_values == rand_test).sum() / len(test_values)
    accs.append(acc)
  return np.mean(accs) + np.std(accs)

In [6]:
tg = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg.parquet')

In [33]:
targets = tg.columns
for company in companies:
  print(company)
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv').date[0]
  comp = tg[tg[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]

  tr = [col for col in targets if company in col]
  if company in tr:
    tr.remove(company)


  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 10)
      if len(lines_train) == 8:
        lines_train = lines_train[1:]

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 10)
      if len(lines_test) == 8:
        lines_test = lines_test[1:]
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_rf_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]
      if len(base_train) == 8:
        base_train = base_train[1:]
    
  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba, t in zip(lines_train, lines_test, base_train, tr):
    best_acc = sample_test(ba, test[t].values.astype(int), n_times=1000)
    if best_acc < te:
      print(t)
      print(f'{round((te-best_acc)*100, 2)} %')

ALRS
ALRS_10_min_mean
2.83 %
ALRS_1_hour_mean
4.8 %
ALRS_1_day_mean
1.42 %
AFLT
VTBR
VTBR_1_min_mean
5.15 %
VTBR_5_min_mean
4.71 %
VTBR_10_min_mean
3.59 %
VTBR_15_min_mean
4.86 %
VTBR_30_min_mean
1.15 %
GAZP
GAZP_1_min_mean
3.55 %
GAZP_5_min_mean
3.05 %
GAZP_10_min_mean
1.56 %
GAZP_15_min_mean
2.32 %
GAZP_30_min_mean
3.5 %
GAZP_1_hour_mean
4.29 %
GAZP_1_day_mean
5.37 %
GMKN
GMKN_1_day_mean
2.03 %
LSRG
DSKY
DSKY_1_min_mean
6.79 %
DSKY_5_min_mean
8.56 %
DSKY_10_min_mean
7.81 %
DSKY_15_min_mean
9.71 %
DSKY_30_min_mean
4.98 %
DSKY_1_hour_mean
3.48 %
IRAO
IRAO_1_min_mean
2.03 %
LKOH
LKOH_1_min_mean
1.36 %
LKOH_1_day_mean
2.23 %
MAGN
MOEX
MOEX_1_min_mean
1.33 %
CBOM
MTSS
ROSN
ROSN_1_min_mean
0.07 %
ROSN_5_min_mean
0.84 %
ROSN_10_min_mean
2.33 %
ROSN_15_min_mean
0.87 %
ROSN_30_min_mean
1.28 %
NLMK
NVTK
NVTK_1_min_mean
1.14 %
NVTK_10_min_mean
0.27 %
NVTK_30_min_mean
0.0 %
PIKK
PLZL
RTKM
RUAL
RUAL_1_hour_mean
0.3 %
HYDR
HYDR_1_min_mean
0.27 %
SBER
SBER_5_min_mean
0.28 %
SBER_10_min_mean
1.52 %


In [34]:
targets = tg.columns
for company in companies:
  print(company)
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv').date[0]
  comp = tg[tg[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]

  tr = [col for col in targets if company in col]
  if company in tr:
    tr.remove(company)


  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 10)
      if len(lines_train) == 8:
        lines_train = lines_train[1:]

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 10)
      if len(lines_test) == 8:
        lines_test = lines_test[1:]
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/tg_loggs/{company}/{company}_boosting_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]
      if len(base_train) == 8:
        base_train = base_train[1:]
    
  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba, t in zip(lines_train, lines_test, base_train, tr):
    best_acc = sample_test(ba, test[t].values.astype(int), n_times=1000)
    if best_acc < te:
      print(t)
      print(f'{round((te-best_acc)*100, 2)} %')

ALRS
AFLT
VTBR
VTBR_1_min_mean
3.56 %
VTBR_5_min_mean
1.79 %
VTBR_10_min_mean
1.38 %
VTBR_15_min_mean
2.66 %
VTBR_30_min_mean
0.69 %
VTBR_1_hour_mean
0.07 %
GAZP
GAZP_1_min_mean
0.78 %
GAZP_5_min_mean
2.18 %
GAZP_10_min_mean
1.77 %
GAZP_15_min_mean
1.29 %
GAZP_30_min_mean
1.9 %
GAZP_1_hour_mean
2.88 %
GAZP_1_day_mean
3.34 %
GMKN
GMKN_1_day_mean
0.19 %
LSRG
DSKY
DSKY_1_min_mean
6.86 %
DSKY_5_min_mean
6.86 %
DSKY_10_min_mean
6.18 %
DSKY_15_min_mean
6.11 %
DSKY_30_min_mean
2.23 %
DSKY_1_hour_mean
3.54 %
IRAO
LKOH
MAGN
MOEX
MOEX_1_min_mean
1.1 %
CBOM
MTSS
ROSN
ROSN_10_min_mean
2.17 %
ROSN_15_min_mean
1.26 %
ROSN_1_hour_mean
0.1 %
NLMK
NVTK
NVTK_1_min_mean
1.13 %
PIKK
PLZL
RTKM
RUAL
RUAL_1_hour_mean
1.07 %
HYDR
SBER
SBER_5_min_mean
0.18 %
SBER_15_min_mean
1.09 %
CHMF
CHMF_1_min_mean
3.77 %
CHMF_5_min_mean
1.9 %
CHMF_10_min_mean
1.4 %
CHMF_15_min_mean
2.59 %
CHMF_30_min_mean
0.34 %
AFKS
AFKS_1_min_mean
1.17 %
SNGS
SNGS_1_day_mean
2.58 %
TATN
TCSG
TCSG_1_min_mean
0.41 %
TCSG_1_day_mean
0.1 %


In [35]:
news = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/news.parquet')

In [37]:
targets = news.columns
for company in companies:
  print(company)
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv').date[0]
  comp = news[news[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]

  tr = [col for col in targets if company in col]
  if company in tr:
    tr.remove(company)


  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 13)
      if len(lines_train) == 8:
        lines_train = lines_train[1:]

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 13)
      if len(lines_test) == 8:
        lines_test = lines_test[1:]
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_rf_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]
      if len(base_train) == 8:
        base_train = base_train[1:]
    
  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba, t in zip(lines_train, lines_test, base_train, tr):
    best_acc = sample_test(ba, test[t].values.astype(int), n_times=1000)
    if best_acc < te:
      print(t)
      print(f'{round((te-best_acc)*100, 2)} %')

ALRS
AFLT
AFLT_1_min_mean
2.29 %
AFLT_5_min_mean
0.93 %
AFLT_10_min_mean
1.34 %
AFLT_15_min_mean
1.58 %
VTBR
VTBR_1_min_mean
3.04 %
VTBR_5_min_mean
3.89 %
VTBR_10_min_mean
4.05 %
VTBR_15_min_mean
4.39 %
VTBR_30_min_mean
4.68 %
VTBR_1_hour_mean
3.83 %
VTBR_1_day_mean
0.18 %
GAZP
GAZP_1_min_mean
1.44 %
GAZP_5_min_mean
2.57 %
GAZP_10_min_mean
1.81 %
GAZP_15_min_mean
1.08 %
GAZP_30_min_mean
1.29 %
GAZP_1_hour_mean
1.42 %
GAZP_1_day_mean
1.94 %
GMKN
GMKN_1_day_mean
0.49 %
LSRG
LSRG_5_min_mean
3.56 %
LSRG_15_min_mean
7.08 %
LSRG_1_hour_mean
5.84 %
LSRG_1_day_mean
5.45 %
DSKY
DSKY_1_min_mean
7.6 %
DSKY_5_min_mean
10.12 %
DSKY_10_min_mean
7.47 %
DSKY_15_min_mean
0.96 %
DSKY_1_day_mean
2.57 %
IRAO
LKOH
LKOH_5_min_mean
1.07 %
LKOH_15_min_mean
0.32 %
LKOH_30_min_mean
0.24 %
MAGN
MAGN_1_min_mean
1.72 %
MAGN_5_min_mean
3.35 %
MAGN_10_min_mean
0.14 %
MAGN_15_min_mean
1.19 %
MAGN_30_min_mean
0.8 %
MAGN_1_hour_mean
1.06 %
MAGN_1_day_mean
3.07 %
MOEX
MOEX_5_min_mean
2.11 %
MOEX_10_min_mean
2.36 %
MOEX_

In [38]:
targets = news.columns
for company in companies:
  print(company)
  first_day = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/{company}.csv').date[0]
  comp = news[news[company] == True].copy()
  comp = comp[comp.date >= first_day].copy()
  comp.dropna(inplace=True)
  train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]

  tr = [col for col in targets if company in col]
  if company in tr:
    tr.remove(company)


  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_train_logg.txt') as f:
      lines = f.readlines()
      lines_train = [float(el) for el in lines]
      lines_train = np.array(lines_train).reshape(-1, 13)
      if len(lines_train) == 8:
        lines_train = lines_train[1:]

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_test_logg.txt') as f:
      lines = f.readlines()
      lines_test = [float(el) for el in lines]
      lines_test = np.array(lines_test).reshape(-1, 13)
      if len(lines_test) == 8:
        lines_test = lines_test[1:]
      inds_max = np.argmax(lines_test, axis=1)

  with open(f'/content/drive/MyDrive/Диссертация/news_loggs/{company}/{company}_boosting_base_acc_logg.txt') as f:
      lines = f.readlines()
      base_train = [float(el) for el in lines]
      if len(base_train) == 8:
        base_train = base_train[1:]
    
  lines_train = np.choose(inds_max, lines_train.T)
  lines_test = np.choose(inds_max, lines_test.T)

  res = []
  for tr, te, ba, t in zip(lines_train, lines_test, base_train, tr):
    best_acc = sample_test(ba, test[t].values.astype(int), n_times=1000)
    if best_acc < te:
      print(t)
      print(f'{round((te-best_acc)*100, 2)} %')

ALRS
ALRS_1_min_mean
0.08 %
AFLT
AFLT_1_min_mean
1.35 %
AFLT_10_min_mean
1.33 %
AFLT_15_min_mean
1.28 %
AFLT_1_day_mean
0.79 %
VTBR
VTBR_1_min_mean
2.38 %
VTBR_5_min_mean
2.91 %
VTBR_10_min_mean
3.41 %
VTBR_15_min_mean
3.67 %
VTBR_30_min_mean
4.07 %
VTBR_1_hour_mean
2.26 %
VTBR_1_day_mean
0.27 %
GAZP
GAZP_1_min_mean
1.39 %
GAZP_5_min_mean
2.62 %
GAZP_10_min_mean
1.51 %
GAZP_15_min_mean
1.16 %
GAZP_30_min_mean
0.77 %
GAZP_1_hour_mean
1.24 %
GAZP_1_day_mean
1.11 %
GMKN
GMKN_1_min_mean
0.1 %
GMKN_10_min_mean
0.3 %
GMKN_1_day_mean
0.7 %
LSRG
LSRG_1_day_mean
0.64 %
DSKY
DSKY_1_min_mean
0.58 %
DSKY_5_min_mean
3.64 %
DSKY_10_min_mean
1.98 %
DSKY_15_min_mean
4.44 %
IRAO
IRAO_5_min_mean
1.03 %
IRAO_10_min_mean
1.28 %
IRAO_30_min_mean
0.02 %
IRAO_1_hour_mean
2.89 %
IRAO_1_day_mean
3.61 %
LKOH
LKOH_1_min_mean
0.12 %
LKOH_5_min_mean
0.09 %
LKOH_1_hour_mean
1.69 %
LKOH_1_day_mean
0.07 %
MAGN
MAGN_30_min_mean
0.98 %
MAGN_1_hour_mean
0.32 %
MAGN_1_day_mean
1.45 %
MOEX
MOEX_1_min_mean
0.3 %
MOEX_5_min